# CS 1656 – Introduction to Data Science  
## Instructor: Alexandros Labrinidis 
## Teaching Assistant: Xiaoting Li
## Additional Credits: Tahereh Arabghalizi, Evangelos Karageorgos, Agha Zuha, Anatoli Shein
## SQLite in Python

In this recitation we will learn how to create SQLite Databases, create tables, populate tables, and execute SQL queries.

Start off by importing sqlite3, which comes installed with Anaconda's package list.

In [17]:
import  sqlite3 as lite

### Introduction to SQLite 
SQLite is an in-process library that implements a self-contained, serverless, zero-configuration, transactional SQL database engine. Unlike most other SQL databases, SQLite does not have a separate server process. SQLite reads and writes directly to ordinary disk files.

### Creating and Connecting to SQLite Database
To connect to a database, use the connect() method which returns a connection object. If a database with that name does not exist, connect() method creates a database.

In [18]:
con = lite.connect('cs1656wed.sqlite')

### Create/Drop Tables & Insert Data
From the connection, we get the cursor object. The cursor is used to traverse the records from the result set. 
By using the with keyword, the Python interpreter automatically releases the resources by closing the connection, provides error handling and __commits__ the changes. Otherwise, each update to the database has to be committed manually. You can think of commit as saving the changes.

We call the execute() method of the cursor to execute the SQL statements.Let's start by creating a Rankings table in the database. 

In [19]:
with con:
    cur = con.cursor() 
    cur.execute('DROP TABLE IF EXISTS Courses')
    cur.execute("CREATE TABLE Courses(cid INT, number INT, professor TEXT, major TEXT, year INT, semester TEXT)")

    cur.execute('DROP TABLE IF EXISTS Majors')
    cur.execute("CREATE TABLE Majors(sid INT, major TEXT)")

    cur.execute('DROP TABLE IF EXISTS Grades')
    cur.execute("CREATE TABLE Grades(sid INT, cid INT, credits INT, grade INT)")

    cur.execute('DROP TABLE IF EXISTS Students')
    cur.execute("CREATE TABLE Students(sid INT, firstName TEXT, lastName TEXT, yearStarted INT)")
   

Now data can be inserted in the table using two ways. You could either insert each row one by one as shown below, 

In [20]:
import pandas
from sqlalchemy import create_engine

engine = create_engine("sqlite:///cs1656wed.sqlite")
df1 = pandas.read_csv('students.csv')
df1.to_sql('students', engine, if_exists='append', index=False)

df2 = pandas.read_csv('grades.csv')
df2.to_sql('grades', engine, if_exists='append', index=False)

df3 = pandas.read_csv('courses.csv')
df3.to_sql('courses', engine, if_exists='append', index=False)

df4 = pandas.read_csv('majors.csv')
df4.to_sql('majors', engine, if_exists='append', index=False)


Or a easier way to insert all rows together is by using executemany() method. But before we try the second method of inserting data, let's first drop the exising table and create it again.

### Select, Where, Orderby
To select all data from the table, 

In [21]:
cur.execute("SELECT * FROM students")

To retrieve data after executing a SELECT statement, you can either treat the cursor as an iterator and call the cursor’s fetchone() method to retrieve a single matching row, or call fetchall() to get a list of the matching rows.

In [22]:
for row in cur.execute("select * from students"):
    print(row)

# OR     
cur.execute("select * from students")
df5 = pandas.DataFrame(cur.fetchall(), columns=[column[0] for column in cur.description])
df5

(555, 'Solange', 'Knowles', 2012)
(666, 'Peter', 'Weiner', 2012)
(1111, 'Ya', 'Boi', 2020)
(1234, 'Michael', 'Scott', 2009)
(1337, 'Beyonce', 'Knowles', 1985)
(1345, 'Julius', 'Caesar', -60)
(1865, 'Abraham', 'Lincoln', 2012)
(3321, 'Mark', 'Brandanowitz', 1992)
(4224, 'Michelle', 'Young', 1984)
(4444, 'Grace', 'Hopper', 1944)
(5376, 'Poverty', 'Jones', 1969)
(5432, 'Mark', 'Wahlberg', 2000)
(6969, 'Thug', 'Nugget', 1862)
(7928, 'John', 'Cash', 1950)
(9191, 'Margaret', 'Mead', 1919)
(9878, 'First', 'Last', 2014)
(9999, 'Elon', 'Musk', 1932)
(14325, 'John', 'Doe', 1999)
(69420, 'Ray', 'Zimmerman', 2017)
(90210, 'Kappa', 'Pride', 2018)
(314158, 'Mr.', 'Pie', 1000)
(999831, 'John', 'Cena', 2003)
(89990, 'BoJack', 'Horseman', 2012)


,sid,firstName,lastName,yearStarted
0,555,Solange,Knowles,2012
1,666,Peter,Weiner,2012
2,1111,Ya,Boi,2020
3,1234,Michael,Scott,2009
4,1337,Beyonce,Knowles,1985
5,1345,Julius,Caesar,-60
6,1865,Abraham,Lincoln,2012
7,3321,Mark,Brandanowitz,1992
8,4224,Michelle,Young,1984
9,4444,Grace,Hopper,1944


Now, let's find out how many courses were passed per semester (plus year)

In [23]:
q3a = """
SELECT year, semester, count(*) 
FROM courses natural join grades 
WHERE grade > 0
GROUP BY year, semester
"""
cur.execute(q3a)
cur.fetchall()

[(-59, 'Fall', 2),
 (-58, 'Fall', 1),
 (1776, 'Summer', 4),
 (1920, 'Fall', 2),
 (1951, 'Spring', 1),
 (1966, 'Summer', 1),
 (1969, 'Spring', 1),
 (1986, 'Summer', 1),
 (1993, 'Spring', 1),
 (1994, 'Fall', 1),
 (1999, 'Spring', 2),
 (2002, 'Fall', 2),
 (2009, 'Spring', 1),
 (2013, 'Fall', 1),
 (2016, 'Fall', 3),
 (2016, 'Spring', 1),
 (2017, 'Fall', 1),
 (2017, 'Spring', 3)]

Let's create a view called 'allgrades' that compiles student grades, and show the view using a dataframe.

In [24]:
cur.execute("DROP VIEW IF EXISTS allgrades")
q4c = """
create view allgrades as
SELECT s.firstName, s.lastName, m.major as ms, 
       c.number, c.major as mc, g.grade 
FROM students as s, majors as m, grades as g, courses as c
WHERE s.sid = m.sid AND g.sid = s.sid AND g.cid = c.cid
"""
cur.execute(q4c)
pandas.DataFrame(cur.execute("select * from allgrades").fetchall(), columns=[column[0] for column in cur.description])

,firstName,lastName,ms,number,mc,grade
0,Peter,Weiner,Women's Studies,8,Administration of Justice,3
1,Peter,Weiner,Women's Studies,13,KappaPriding,3
2,Peter,Weiner,Women's Studies,1567,MILT,4
3,Peter,Weiner,Women's Studies,1111,Women's Studies,0
4,Ya,Boi,Underwater Basket Weaving,420,CS,3
5,Ya,Boi,Underwater Basket Weaving,1113,Underwater Basket Weaving,4
6,Ya,Boi,Underwater Basket Weaving,2011,Underwater Basket Weaving,4
7,Michael,Scott,Paper Supplies,1,Paper,1
8,Julius,Caesar,Classics,1568,MILT,4
9,Julius,Caesar,Classics,1567,MILT,3


### Tasks

__T1) Show how many courses were passed (grade>0) per student per semester (plus year). Show student id, year, semester and the count.__

__T2) Same as T1, but show student first and last name instead of student id. Also only show results for students passing at least two courses for every semester.__

__T3) Show the students that have failed at a course in their majors (firstName, lastName, major, courseNumber), utilizing the 'allgrades' view.__

__T4) Same as T3, but without utilizing the view.__

__T5) Show the professors in decreasing order of 'success' (professor, success). Success will be defined as the number of students passing any of the courses with grade >= 2.__

__T6) Show a report of the courses (course_number, student_names, avg_grade). Column 'student_names' will contain the first and last names (seperated by a space) of all students taking the course, each name being seperated by ', ' (eg. 'John Doe, Mary Jane'). Only students that passed a specific course (grade>=2) will be considered. Also, the report should only contain courses with avg_grade > 3.__

In [25]:
# task1

a1 = """
SELECT s.sid, year, semester, count(*) 
FROM students as s, courses natural join grades 
WHERE grade > 0
GROUP BY s.sid, semester
"""
cur.execute(a1)
cur.fetchall()


[(666, -59, 'Fall', 1),
 (666, 2017, 'Spring', 1),
 (666, 1776, 'Summer', 1),
 (1111, 2017, 'Fall', 2),
 (1111, 2016, 'Spring', 1),
 (1234, 2009, 'Spring', 1),
 (1345, -59, 'Fall', 2),
 (1865, 2013, 'Fall', 1),
 (3321, 1993, 'Spring', 1),
 (4224, 1986, 'Summer', 1),
 (4444, 1920, 'Fall', 3),
 (4444, 1999, 'Spring', 2),
 (5432, 2002, 'Fall', 1),
 (5432, 1776, 'Summer', 1),
 (6969, 1969, 'Spring', 1),
 (7928, 1951, 'Spring', 1),
 (9191, 1920, 'Fall', 1),
 (9191, 1999, 'Spring', 1),
 (9878, 1966, 'Summer', 1),
 (9999, 1776, 'Summer', 1),
 (89990, 1776, 'Summer', 1),
 (90210, 2017, 'Spring', 1),
 (314158, 1994, 'Fall', 1),
 (999831, 2002, 'Fall', 1)]

In [26]:
# task2

a2 = """
SELECT s.firstName, s.lastName, year, semester, count(*) 
FROM students as s, courses natural join grades 
WHERE grade > 0
GROUP BY s.firstName, s.lastName, semester
HAVING count(*) >= 2
"""
cur.execute(a2)
cur.fetchall()


[('Grace', 'Hopper', 1920, 'Fall', 3),
 ('Grace', 'Hopper', 1999, 'Spring', 2),
 ('Julius', 'Caesar', -59, 'Fall', 2),
 ('Ya', 'Boi', 2017, 'Fall', 2)]

In [27]:
# task3
cur.execute("DROP VIEW IF EXISTS allgrades")
q4c = """
create view allgrades as
SELECT s.firstName, s.lastName, m.major as ms, 
       c.number
FROM students as s, majors as m, grades as g, courses as c
WHERE s.sid = m.sid AND g.sid = s.sid AND g.cid = c.cid AND m.major = c.major
"""
cur.execute(q4c)
pandas.DataFrame(cur.execute("select * from allgrades").fetchall(), columns=[column[0] for column in cur.description])

,firstName,lastName,ms,number
0,Peter,Weiner,Women's Studies,1111
1,Ya,Boi,Underwater Basket Weaving,1113
2,Ya,Boi,Underwater Basket Weaving,2011
3,Julius,Caesar,MILT,1568
4,Julius,Caesar,MILT,1567
5,Abraham,Lincoln,Theatre,4
6,Mark,Brandanowitz,Urban Planning,2
7,Michelle,Young,Film Study,2000
8,Grace,Hopper,Anthropology,1313
9,Grace,Hopper,Anthropology,73652


In [28]:
# task4
q4c = """
SELECT s.firstName, s.lastName, m.major as ms, 
       c.number
FROM students as s, majors as m, grades as g, courses as c
WHERE s.sid = m.sid AND g.sid = s.sid AND g.cid = c.cid AND m.major = c.major
"""
cur.execute(q4c)
cur.fetchall()

[('Peter', 'Weiner', "Women's Studies", 1111),
 ('Ya', 'Boi', 'Underwater Basket Weaving', 1113),
 ('Ya', 'Boi', 'Underwater Basket Weaving', 2011),
 ('Julius', 'Caesar', 'MILT', 1568),
 ('Julius', 'Caesar', 'MILT', 1567),
 ('Abraham', 'Lincoln', 'Theatre', 4),
 ('Mark', 'Brandanowitz', 'Urban Planning', 2),
 ('Michelle', 'Young', 'Film Study', 2000),
 ('Grace', 'Hopper', 'Anthropology', 1313),
 ('Grace', 'Hopper', 'Anthropology', 73652),
 ('Grace', 'Hopper', 'CS', 2),
 ('Grace', 'Hopper', 'CS', 420),
 ('Grace', 'Hopper', 'CS', 1656),
 ('Mark', 'Wahlberg', 'Communications', 101),
 ('John', 'Cash', 'Music', 101),
 ('Margaret', 'Mead', 'Anthropology', 1313),
 ('Margaret', 'Mead', 'Anthropology', 73652),
 ('Kappa', 'Pride', 'KappaPriding', 13),
 ('John', 'Cena', "U Can't See Me", 15)]

In [32]:
# task5

q4c = """
SELECT c.professor,  count(*) as success
FROM grades as g, courses as c
WHERE  g.cid = c.cid AND g.grade >= 2
group by professor
"""
cur.execute(q4c)
cur.fetchall()

[('A.T. Great', 3),
 ('Adam J. Lee', 2),
 ('Bo Jangles', 1),
 ('Doctor Professor Cosmo Kramer Esquire', 1),
 ('Donnie', 1),
 ('Drew Jackson', 1),
 ('Flitwick', 2),
 ('Franz Boas', 2),
 ('John Smith', 1),
 ('Kirk Pruhs', 1),
 ('L. Knope', 1),
 ('Labrinidis', 1),
 ('Michael Santana', 2),
 ('Misurda', 3),
 ('Satan', 1),
 ('Seventy', 1),
 ('Sun', 1),
 ('Vince McMahon', 1)]

In [39]:
# task5
q4c = """
SELECT   c.number, AVG(g.grade)
FROM students as s,grades as g, courses as c
WHERE  g.sid = s.sid AND g.cid = c.cid AND g.grade >= 2
GROUP BY c.number
HAVING AVG(g.grade) > 3
"""
cur.execute(q4c)
cur.fetchall()


[(2, 4.0),
 (8, 3.3333333333333335),
 (13, 3.5),
 (15, 4.0),
 (245, 4.0),
 (420, 3.5),
 (1069, 4.0),
 (1113, 4.0),
 (1567, 3.5),
 (1568, 4.0),
 (1656, 4.0),
 (1999, 4.0),
 (2011, 4.0),
 (73652, 4.0)]

In [15]:
cur.close()
con.close()